In [11]:
from google.colab import drive
drive.mount('/content/drive')
import os
!pip install opencv-python
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
# Change the current working directory
os.chdir('/content/drive/My Drive/CV_Assignment4')

# Verify the current working directory
print("Current directory:", os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/drive/My Drive/CV_Assignment4


In [12]:
from google.colab import drive
import numpy as np
import cv2


# Open the video file from your Google Drive
video_path = '/content/drive/My Drive/CV_Assignment4/vtest.avi'
cap = cv2.VideoCapture(video_path)

# Check if the video was opened successfully
if not cap.isOpened():
    raise IOError("Cannot open the video file. Check the file path and permissions.")

# Read the first frame to get the video properties
ret, frame = cap.read()
if not ret:
    raise IOError("Cannot read from the video file. Check if the file is corrupted or in the correct format.")

height, width = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS)

# Create a VideoWriter object to write the video
output_video_path = '/content/drive/My Drive/CV_Assignment4/output.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=200, qualityLevel=0.1, minDistance=7, blockSize=7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.02))

# Initialize tracking points list
tracks = []
track_len = 15
detect_interval = 5
frame_idx = 0

motion_threshold = 1.0  # Minimum distance to consider motion

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    vis = frame.copy()

    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255
        for x, y in [np.int32(tr[-1]) for tr in tracks]:
            cv2.circle(mask, (x, y), 5, 0, -1)
        p = cv2.goodFeaturesToTrack(frame_gray, mask=mask, **feature_params)
        if p is not None:
            for x, y in np.float32(p).reshape(-1, 2):
                tracks.append([(x, y)])

    if len(tracks) > 0:
        p0 = np.float32([tr[-1] for tr in tracks]).reshape(-1, 1, 2)
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None, **lk_params)
        p0r, st0, err0 = cv2.calcOpticalFlowPyrLK(frame_gray, prev_gray, p1, None, **lk_params)
        d = abs(p0 - p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_tracks = []
        for tr, (x, y), good_flag in zip(tracks, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            last_tracked_point = np.array(tr[-1])
            new_point = np.array((x, y))
            if np.linalg.norm(last_tracked_point - new_point) < motion_threshold:
                continue
            tr.append((x, y))
            if len(tr) > track_len:
                del tr[0]
            new_tracks.append(tr)
            cv2.circle(vis, (int(x), int(y)), 2, (0, 255, 0), -1)
        tracks = new_tracks
        cv2.polylines(vis, [np.int32(tr) for tr in tracks], False, (0, 255, 0))

    frame_idx += 1
    prev_gray = frame_gray

    # Write the processed frame to the output video
    out.write(vis)

# Release resources
cap.release()
out.release()

print(f"Output video is saved as {output_video_path}")


Output video is saved as /content/drive/My Drive/CV_Assignment4/output.avi


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output video is saved as /content/drive/My Drive/CV_Assignment4/output_tracked.avi
